In [1]:
# criar o navegador
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico)

In [5]:
# abrir a página index (entrar no site da busca jurídica)
import os

caminho = os.getcwd()
arquivo = caminho + r"\index.html"


In [6]:
print(arquivo)

C:\Users\leand\OneDrive\Documentos\Curso Hashtag impressionador (Lira)\Hashtag_Impressionador\Selenium\Projeto Site Pesquisa Juridica\index.html


In [3]:
# importar a base de dados
import pandas as pd

tabela = pd.read_excel("Processos.xlsx")
display(tabela)

,Nome,Advogado,Processo,Cidade,Status
0,Lira,Alon Lawyer,PC6592,Distrito Federal,NaN
1,João,Lawyer Alon,EB3792,Rio de Janeiro,NaN
2,Amanda,Amanda mesmo,MM1043,Rio de Janeiro,NaN
3,Carol,Amanda,PC5197,São Paulo,NaN


In [4]:
from selenium.webdriver import ActionChains
import time

for linha in tabela.index:

    # para cada processo (linha da tabela)
    navegador.get(arquivo)

    # abrir a lista de cidades
    botao = navegador.find_element(By.CLASS_NAME, 'dropdown-menu')
    ActionChains(navegador).move_to_element(botao).perform()
    
    cidade = tabela.loc[linha, "Cidade"]
    
    # selecionando o cidade
    navegador.find_element(By.PARTIAL_LINK_TEXT, cidade).click()
    
    # mudar para a nova aba
    aba_original = navegador.window_handles[0]
    indice = 1 + linha
    nova_aba = navegador.window_handles[indice]
    
    navegador.switch_to.window(nova_aba)
    
    # preencher o formulário com os dados de busca
    navegador.find_element(By.ID, 'nome').send_keys(tabela.loc[linha, "Nome"])
    navegador.find_element(By.ID, 'advogado').send_keys(tabela.loc[linha, "Advogado"])
    navegador.find_element(By.ID, 'numero').send_keys(tabela.loc[linha, "Processo"])

    # clicar em pesquisar
    navegador.find_element(By.CLASS_NAME, 'registerbtn').click()
    
    # confirmar a pesquisa
    alerta = navegador.switch_to.alert
    alerta.accept()
    
    # esperar o resultado da pesquisa e agir de acordo com o resultado
    while True:
        try:
            alerta = navegador.switch_to.alert
            break
        except:
            time.sleep(1)
    texto_alerta = alerta.text

    if "Processo encontrado com sucesso" in texto_alerta:
        alerta.accept()
        tabela.loc[linha, "Status"] = "Encontrado"
    else:
        tabela.loc[linha, "Status"] = "Não encontrado"
        alerta.accept()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"partial link text","selector":"São Paulo"}
  (Session info: chrome=104.0.5112.80)
Stacktrace:
Backtrace:
	Ordinal0 [0x005C78B3+2193587]
	Ordinal0 [0x00560681+1771137]
	Ordinal0 [0x004741A8+803240]
	Ordinal0 [0x004A24A0+992416]
	Ordinal0 [0x004A273B+993083]
	Ordinal0 [0x004CF7C2+1177538]
	Ordinal0 [0x004BD7F4+1103860]
	Ordinal0 [0x004CDAE2+1170146]
	Ordinal0 [0x004BD5C6+1103302]
	Ordinal0 [0x004977E0+948192]
	Ordinal0 [0x004986E6+952038]
	GetHandleVerifier [0x00870CB2+2738370]
	GetHandleVerifier [0x008621B8+2678216]
	GetHandleVerifier [0x006517AA+512954]
	GetHandleVerifier [0x00650856+509030]
	Ordinal0 [0x0056743B+1799227]
	Ordinal0 [0x0056BB68+1817448]
	Ordinal0 [0x0056BC55+1817685]
	Ordinal0 [0x00575230+1856048]
	BaseThreadInitThunk [0x7709FA29+25]
	RtlGetAppContainerNamedObjectPath [0x773E7A9E+286]
	RtlGetAppContainerNamedObjectPath [0x773E7A6E+238]


In [ ]:
navegador.quit()
display(tabela)

tabela.to_excel("Processos Atualizado.xlsx")